In [ ]:
# Constants
SEED = 33
TEMP = 0
TIMEOUT = 5000

# Define Prompt
PROMPT = "Create an Expert Advisor for MT5 that identifies key entry points using ICT concepts"

In [ ]:
%pip install pyautogen~=0.1.0 langchain duckduckgo-search

In [ ]:
import autogen
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

config_list = [
    {
        "model": "gpt-4-1106-preview",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-4",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-4-0314",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-4-0613",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo-1106",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo-16k-0613",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo-instruct",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo-instruct-0914",
        "api_key": OPENAI_API_KEY,
    },
    {
        "model": "gpt-3.5-turbo",
        "api_key": OPENAI_API_KEY,
    },
]

In [ ]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.agents.agent_toolkits import FileManagementToolkit
import os

# Get the current working directory
current_dir = os.getcwd()

# Define discovery output folder
working_directory = os.path.join(current_dir, "output/discovery/")

# Create Langchain toolkit
tools = FileManagementToolkit(
    root_dir=str(working_directory),
).get_tools()

copy_tool, delete_tool, search_tool, move_tool, read_tool, write_tool, list_tool = tools

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [ ]:
# Define a function to generate llm_config from a LangChain tool
def generate_llm_config(tool):
    # Define the function schema based on the tool's args_schema
    function_schema = {
        "name": tool.name.lower().replace (' ', '_'),
        "description": tool.description,
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
        },
    }

    if tool.args is not None:
      function_schema["parameters"]["properties"] = tool.args

    return function_schema

# Construct the llm_config
llm_config = {
  "seed": SEED,
  "temperature": TEMP,
  "request_timeout": 500,
  "max_retry_period": 500,
  "retry_wait_time": 500,
  "config_list": config_list,
  "functions":[
      generate_llm_config(copy_tool),
      generate_llm_config(delete_tool),
      generate_llm_config(search_tool),
      generate_llm_config(move_tool),
      generate_llm_config(read_tool),
      generate_llm_config(write_tool),
      generate_llm_config(list_tool),
      generate_llm_config(search_tool),
  ],
}

In [ ]:
# Agents

# Admin
# A human admin. Interact with the Planner to discuss plan. Plan execution needs to be approved by this admin. 
admin = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the Planner to discuss plan. Plan execution needs to be approved by this admin.",
   llm_config=llm_config,
   human_input_mode="TERMINATE",
   code_execution_config=False,
   function_map={
     copy_tool.name: copy_tool._run,
     delete_tool.name: delete_tool._run,
     search_tool.name: search_tool._run,
     move_tool.name: move_tool._run,
     read_tool.name: read_tool._run,
     write_tool.name: write_tool._run,
     list_tool.name: list_tool._run,
     search_tool.name: search_tool._run,
   },
)

# Planner
# Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
# Break down the larger task into smaller features and assign to members of your team.
# The plan may involve a CodingExpert who can write code or technical documentation and a TradingExpert who can create trading strategies.
# Explain the plan first. Be clear which step is performed by the CodingExpert, and which step is performed by a TradingExpert.
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
Break down the larger task into smaller features and assign to members of your team.
The plan may involve a CodingExpert who can write code and a TradingExpert who can create trading strategies.
Explain the plan first. Be clear which step is performed by the CodingExpert, and which step is performed by a TradingExpert.
""",
    llm_config=llm_config,
)

# TradingExpert
# TradingExpert. Create a trading strategy or technical indicator based on the requirements provided by the Planner. Once approved by the Planner, write the strategy to a new text file.
trading_expert = autogen.AssistantAgent(
    name="TradingExpert",
    system_message="TradingExpert. Create a trading strategy or technical indicator based on the requirements provided by the Planner.",
    llm_config=llm_config,
)

# CodingExpert
# CodingExpert. Create functional requirements for the engineering team based on the strategy provided by the TradingExpert. Once approved by the Planner, write the functional requirements to a new text file.
coding_expert = autogen.AssistantAgent(
    name="CodingExpert",
    system_message="CodingExpert. Write the code based on the requorements provided by the Planner.",
    llm_config=llm_config,
)

# Critic
# Critic. Double check plan, claims, code from other agents and provide feedback. Review trading strategy to ensure it aligns with projet requirements. Review functional requirements to ensure it aligns with trading strategy.
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Review trading strategy to ensure it aligns with projet requirements.",
    llm_config=llm_config,
)

In [ ]:
# Group Chat & Manager
groupchat = autogen.GroupChat(agents=[admin, planner, trading_expert, coding_expert, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
# Initiate Chat
admin.initiate_chat(
  manager,
  message=PROMPT,
  clear_history=False
)